# 文件导入与处理

```
author: yudake
date: 2/18/2018
```

本项目是利用 **文本卷积神经网络（TextCNN）** 对文章进行二分类，以区别文章是否为色情小说。

## 一、导入将要使用的库

- numpy：Anconda环境下自带
- pandas：Anconda环境下自带
- jieba：用来进行分词，需自行安装
- pickle：用来进行存储处理后的数据，需自行安装

In [1]:
import numpy as np
import pandas as pd
import jieba
import pickle

## 二、导入数据

一共有23748条数据，其中11999条是正面数据（也就是色情小说），11749条是反面数据（也就是非色情小说）。

可以看出，正反数据接近1:1。

In [2]:
data = pd.read_csv('data/data.csv', encoding='utf-8')
data.head()

,label,fiction
0,1,人妻老师陈雅菲，今年二十八岁，毕业于香港大学英文系，之后就在北区一所中学教英文，这里的学生成...
1,1,（八）艳艳悠悠醒来，在一片迷惘中，忽觉大腿上紧贴着个热呼呼、硬梆梆的棍状物体，同时胸前玉峰上...
2,1,「人……你……你还没要泄吗……淫妇……我……啊……爽死人家了……啊……啊……又要泄了……不行...
3,1,第十九篇艳妓貂蝉36（三十六）貂蝉的房内正春色无边，只见她身上只穿一件银红蝉翼纱衫，内衬贴肉...
4,1,我的肉棒仍然处于亢奋状态，李平儿显然觉察到了，说：“不公平，你还没出来呢！”“我知道。”我说...


In [3]:
data.dropna(axis=0, inplace=True)
data.reset_index(inplace=True, drop=True)
data.shape

(23748, 2)

In [4]:
data[data['label']==1].shape

(11999, 2)

In [5]:
data[data['label']==0].shape

(11749, 2)

## 三、去除特殊符号

可以发现，文章内有很多特殊符号，这是网络文学普遍拥有的特征。除此之外，一篇小说的标点符号也会很多。标点符号对于一篇小说的分类其实是有用的，但是本身我们的小说就是中文，需要进行分词处理，如果再考虑标点符号，对于处理速度影响较大。所有，我们这里准备把所有的符号都删除掉。

In [6]:
def strclear(text,newsign='！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.'):
    import string # 引入string模块
    signtext = string.punctuation + newsign # 引入英文符号常量，可附加自定义字符，默认为空
    signrepl = '@'*len(signtext) # 引入符号列表长度的替换字符
    signtable = str.maketrans(signtext,signrepl) # 生成替换字符表
    return text.translate(signtable).replace('@','') # 最后将替换字符替换为空即可

In [7]:
data['fiction'] = data['fiction'].apply(strclear)
data.head()

,label,fiction
0,1,人妻老师陈雅菲今年二十八岁毕业于香港大学英文系之后就在北区一所中学教英文这里的学生成绩也不很...
1,1,八艳艳悠悠醒来在一片迷惘中忽觉大腿上紧贴着个热呼呼硬梆梆的棍状物体同时胸前玉峰上还有一股沉甸...
2,1,人你你还没要泄吗淫妇我啊爽死人家了啊啊又要泄了不行了人淫妇再下去会会被你的大肉棒插死的啊真要...
3,1,第十九篇艳妓貂蝉36三十六貂蝉的房内正春色无边只见她身上只穿一件银红蝉翼纱衫内衬贴肉小嵌肩下...
4,1,我的肉棒仍然处于亢奋状态李平儿显然觉察到了说不公平你还没出来呢我知道我说还有貂蝉呢貂蝉现在一...


## 四、分词与转换

首先用jieba进行分词处理，然后将分词转换为数值型变量，组成句子向量。

因为一篇小说的词汇量长度过于巨大，而我们转换后的所有句子向量需要拥有相同的长度，对于电脑的存储空间和计算性能要求过高。

我们截取1500个词构成我们的句子向量，1500个词足够检测出文章是不是色情小说。

- 如果本条数据大于3000个词，那么我们截取中间的1500个词；
- 如果本条数据不够3000个词，但是在1500个词以上，我们截取最后1500个词；
- 如果本条数据不够1500个词，那么我们需要对句子向量进行补全，把句子向量补全到1500个词。

In [8]:
for i in range(len(data)):
    data.loc[i, 'fiction'] = ' '.join(jieba.cut(data.loc[i, 'fiction']))


word_set = set()
for i in range(len(data)):
    val = data.loc[i, 'fiction'].split()
    if len(val) > 3000:
        val = val[int(len(val)/2)-750:int(len(val)/2)+750]
    elif len(val) > 1500:
        val = val[int(len(val))-1500:]
    data.loc[i, 'fiction'] = ' '.join(val)
    word_set.update(val)


word_set.add('<PAD>')
word2int = {val:ii for ii, val in enumerate(word_set)}

word_length = 1500
fiction_map = {val:[word2int[row] for row in val.split()] for ii,val in enumerate(set(data['fiction']))}
for key in fiction_map:
    for cnt in range(word_length - len(fiction_map[key])):
        fiction_map[key].insert(len(fiction_map[key]) + cnt,word2int['<PAD>'])

data['fiction'] = data['fiction'].map(fiction_map)
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yudake\AppData\Local\Temp\jieba.cache
Loading model cost 1.094 seconds.
Prefix dict has been built succesfully.


,label,fiction
0,1,"[299817, 333254, 375354, 197126, 117518, 39277..."
1,1,"[333254, 208953, 392779, 8390, 284454, 87704, ..."
2,1,"[102411, 147309, 68438, 333254, 113747, 284454..."
3,1,"[158166, 397736, 284263, 47452, 270648, 217198..."
4,1,"[165651, 63334, 39360, 447183, 366531, 223738,..."


In [9]:
pickle.dump((data), open('data_processed/data_processed.p', 'wb'))
pickle.dump((word_set), open('data_processed/word_set.p', 'wb'))